In [1]:
import numpy as np
import pandas as pd

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import matplotlib
%matplotlib inline

from custom_functions import *

In [2]:
dsa_accme_df = pd.read_csv('./data/energy_analysis/dsa_conv_accme_parameters.csv')
dsa_accme_df = generate_accme_cycles_column(dsa_accme_df)

In [3]:
def accme_ee_cycles(df, cores, config_choice):

    df['Mem Overlap'] = 1 # Set mem overlap so that AccMe returns full mem cycles (Assume non are hidden) 
    df['Cores'] = cores
    df = generate_accme_cycles_column(df)
    compute_cycles = df['AccMe Cycles'][config_choice][0]
    memory_cycles = df['AccMe Cycles'][config_choice][1] # Independent of whether or not they are hidden.

    return compute_cycles, memory_cycles

In [4]:
def get_accme_ee_range(df, cores_array, frequency, core_dpower, core_spower, mem_dpower, mem_spower, config_choice):
    
    aios_per_joule_array = []
    
    for i in range(len(cores_array)):
        cores = cores_array[i]
        compute_cycles, memory_cycles = accme_ee_cycles(df, cores_array[i], config_choice)
        time = max(compute_cycles,memory_cycles) / frequency        

        # Caculate compute static energy
        compute_static_energy = time * core_spower * cores_array[i]
        
        # See if cores are bottlenecked by memory
        core_scaler = compute_cycles / memory_cycles
        effective_cores = cores_array[i] * core_scaler
        if effective_cores < cores_array[i]:
            cores = effective_cores
        
        # Calculate compute dynamic energy
        compute_dynamic_energy = time * core_dpower * cores
        # Memory static energy
        memory_static_energy = time * mem_spower

        # Memory dynamic energy
        memory_dynamic_energy = df['Ext Data Movement'][config_choice] * mem_dpower
        
        # AIO efficiency
        total_energy = compute_static_energy + compute_dynamic_energy + memory_static_energy + memory_dynamic_energy
        aios_per_joule = df['Image Pixels'][config_choice] / total_energy
        aios_per_joule = df['Num AIOs'][config_choice] / total_energy
        aios_per_joule_array.append(aios_per_joule)
    
    return aios_per_joule_array

In [6]:
def plot_aio_energy_efficiency(core_dpower, core_spower, mem_dpower, mem_spower):
    
    linewidth = 4
    accme_alpha = 0.6

    # Figure properties
    fig, (ax1) = plt.subplots(1, 1)
    
    frequency_scaling = 1000000
    core_min = 1
    core_max = 128
    cores_array = np.linspace(core_min,core_max,core_max)
    x = cores_array

    config_choices = [20,21,22,23]
    for i in range(len(config_choices)):
        y = get_accme_ee_range(dsa_accme_df, cores_array, 1200, core_dpower, core_spower, mem_dpower, mem_spower, config_choices[i])
        ax1.plot(x, np.asarray(y)/frequency_scaling, linestyle='-', linewidth=linewidth, alpha=accme_alpha)

    x_fontsize = 16
    y_fontsize = 16
    titlesize = 16
    tick_size = 16
    #legendsize = 16
    subtitle_offset = -0.55
    
    fig.set_figwidth(5)
    fig.set_figheight(3)
    ax1.set_xlabel('Core Count', fontsize = x_fontsize) # X label
    ax1.set_ylabel('AIOs/ Joule', fontsize = y_fontsize) # X label
    ax1.tick_params(axis='both', which='major', labelsize=tick_size)
    ax1.set_title("Convolution", fontsize = titlesize , y=subtitle_offset, fontweight="bold")
    ax1.yaxis.offsetText.set_size(tick_size)
    
    ax1.grid()
    plt.show()

interact(
    plot_aio_energy_efficiency, 
    core_dpower = widgets.FloatSlider(value=0.5, min=0, max=2, step=0.1),
    core_spower = widgets.FloatSlider(value=0.2, min=0, max=0.5, step=0.01),
    mem_dpower = widgets.FloatSlider(value=0.001, min=0, max=0.01, step=0.0001),
    mem_spower = widgets.FloatSlider(value=2, min=0, max=10, step=0.1)  
)

interactive(children=(FloatSlider(value=0.5, description='core_dpower', max=2.0), FloatSlider(value=0.2, descr…

<function __main__.plot_aio_energy_efficiency(core_dpower, core_spower, mem_dpower, mem_spower)>